# **FOREWORD**

This kernel continues from the below kernels- <br>
- https://www.kaggle.com/code/ravi20076/mcts2024-packages-v2 <br>
- https://www.kaggle.com/code/ravi20076/mcts2024-lama-train-v1 <br>

These kernels download packages for the competition and train the model used for the inference kernel here. <br>
The dataset [here](https://www.kaggle.com/datasets/ravi20076/mcts2024baselinev1) loads the files from my training kernel (privately from my local sources). If you wish to use this in your work, you need to fork my above 2 kernels and run them appropriately. Then instead of the dataset, use the training kernel outputs here as input. <br>
We have a variable here called **ip_path**. This needs to be adjusted based on your input path <br>

Best of luck!

# **IMPORTS**

In [ ]:
%%capture

!pip install -q lightautoml --no-index --find-links=/kaggle/input/mcts2024-packages-v2/LAMA

In [ ]:
%%time 

import os
ip_path = f"/kaggle/input/mcts2024baselinev1/LAMAV1_1"

exec(
    open(
        os.path.join(ip_path, 'myimports.py'),'r'
    ).read()
)
exec(
    open(os.path.join(ip_path, 'fe.py'),'r').read()
)

models   = joblib.load(os.path.join(ip_path, f"FittedModels_LAMAV1_1.joblib"))
cat_cols = joblib.load(os.path.join(ip_path, f"CatCols.joblib"))
num_cols = joblib.load(os.path.join(ip_path, f"NumCols.joblib"))

pp = Preprocessor()

print()
collect();

# **SUBMISSION**

In [ ]:
try:
    import kaggle_evaluation.mcts_inference_server
except:
    pass


counter = 0

def postprocesspreds(ypred):
    "This function post-processes the prediction array to clip it between -1 to 1"
    return np.clip(ypred, a_min = -1.0, a_max = 1.0)

def predict(
    test       : pl.DataFrame, 
    submission : pl.DataFrame,
):
    """
    This function predicts the test set using the API-based function using the models trained
    We post-process the test predictions using minima-maxima values based on the values in the train data
    """
    
    global counter, models
    
    Xtest, _, _ = \
    pp.pp_data(
        test,
        split_agent_ftre = True,
        cat_cols = cat_cols,
        num_cols = num_cols,
        label  = 'Test',
    )
    
    test_preds = models.predict(Xtest).data.flatten()
    counter    = counter + 1
        
    return submission.with_columns(
        pl.Series('utility_agent1', postprocesspreds(test_preds))
    )

In [ ]:
%%time 

inference_server = \
kaggle_evaluation.\
mcts_inference_server.\
MCTSInferenceServer(predict)

if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    inference_server.serve()
    
else:
    inference_server.run_local_gateway(
        (
            '/kaggle/input/um-game-playing-strength-of-mcts-variants/test.csv',
            '/kaggle/input/um-game-playing-strength-of-mcts-variants/sample_submission.csv'
        )
    )
    
print()
collect();